In [ ]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
from src.CCLE_postp_function import *
sys.path.insert(0, '../JKBio/')
import Datanalytics as da 
import TerraFunction as terra
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from taigapy import TaigaClient
from IPython.display import Image,display
tc = TaigaClient()
import numpy as np
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

In [ ]:
samplesetname = "19Q4"
prevname="19Q3"
prevprevname ='19Q2'
virtual_public='public-19q4-93d9'
virtual_dmc='dmc-19q4-d5f3'
virtual_internal='internal-19q4-fb11'

workspace1="broad-genomics-delivery/Getz_IBM_CellLines_Exomes"
workspace2="broad-firecloud-ccle/CCLE_DepMap_WES"
workspace3="broad-genomics-delivery/CCLE_DepMap_WES"
refworkspace="broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline"
source1="ibm"
source2="ccle"
source3="ccle"
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"
release = samplesetname

In [ ]:
%%R
release <- '19Q4'
prevname <- '19Q3'
genome_version <- 'hg19'

# Processing release {{samplesetname}}

In [ ]:
wm1 = dm.WorkspaceManager(workspace1)
wm2 = dm.WorkspaceManager(workspace2)
wm3 = dm.WorkspaceManager(workspace3)

refwm = dm.WorkspaceManager(refworkspace)

In [ ]:
extract_to_change = {'ref_bams':'WES_bam'}

In [ ]:
newsamples = createDatasetWithNewCellLines(refwm, samplesetname, [wm1], [source1], match='ACH', participantslicepos=10, accept_unknowntypes=True, gsfolderto="gs://ccle_bams/wes/",extract=extract_to_change,
                                          dry_run=True)

In [ ]:
newsamples = createDatasetWithNewCellLines(refwm, samplesetname, [wm1, wm2, wm3], [source1, source2, source3], match='ACH', participantslicepos=10, accept_unknowntypes=True, gsfolderto="gs://ccle_bams/wes/",extract=extract_to_change,
                                          dry_run=True)

## update all sample set

In [ ]:
refwm.update_sample_set('All_samples',refwm.get_samples().index.tolist())

## Check that we have all the cell lines we expect for this release
This involves comparing to the list in the Google sheet "Cell Line Profiling Status."

In [ ]:
# this function may not work - it hasn't been tested
url = 'https://docs.google.com/spreadsheets/d/1qus-9TKzqzwUMNWp8S1QP4s4-3SsMo2vuQRZrNXf7ag/edit?ts=5db85e27#gid=0&fvid=1627883727'

compareToCuratedGS(url, sample = newsample[0], samplesetname = samplesetname, colname = 'CN New to internal')

# Run the pipeline

In [ ]:
submission_id = refwm.create_submission("CGA_WES_CCLE_Characterization_Pipeline_v0.1_Jul2019", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refworkspace, submission_id)

In [ ]:
submission_id = refwm.create_submission("common_variant_filter", samplesetname, 'sample_set', expression='this.samples')
terra.waitForSubmission(refworkspace, submission_id)

In [ ]:
submission_id = refwm.create_submission("filterMAF_on_CGA_pipeline", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refworkspace, submission_id)

### filtered

In [ ]:
submission_id1 = refwm.create_submission("aggregateMAFs_selectFields_copy", samplesetname)

### unfiltered

In [ ]:
submission_id2 = refwm.create_submission("aggregateMAFs_selectFields_unfiltered", samplesetname)

In [ ]:
terra.waitForSubmission(refworkspace, [submission_id1,submission_id2])

### Save the workflow configurations used

In [ ]:
terra.saveConfigs(refworkspace,'./data/'+samplesetname+'/Mutconfig')

# Downloading from terra

In [ ]:
res = refwm.get_sample_sets().loc[release]
res

In [ ]:
filtered = res['filtered_CGA_MAF_aggregated']
unfiltered = res['unfiltered_CGA_MAF_aggregated']

! gsutil cp $filtered "temp/mutation_filtered_terra_merged.txt"
! gsutil cp $unfiltered "temp/mutation_unfiltered_terra_merged.txt"

###### retrieving RNAseq MAFs

# postprocessing

In [ ]:
a = pd.read_csv('temp/mutation_filtered_terra_merged.txt',sep='\t')

In [ ]:
a.Tumor_Sample_Barcode = [i.split('_') for i in a.Tumor_Sample_Barcode]

In [ ]:
a.to_csv('temp/mutation_filtered_terra_merged.txt',sep='\t')

In [ ]:
%%R
source('../gkugener/RScripts/load_libraries_and_annotations.R')
load('../gkugener/DM_OMICS/Annotations.RData') 
# There are some cell lines the celllinemapr does not know how to map so we need to load this data object for now (from old datasets)
source('src/CCLE_postp_function.R')
library(tidyverse)
library(data.table)
library(magrittr)
library(taigr)
library(cdsomics)
library(celllinemapr) # To pull out DepMap_IDs from CCLE_names where needed

In [ ]:
%%R
newly_merged_maf <- readMutations('temp/mutation_filtered_terra_merged.txt')
new_release <- createSNPs(newly_merged_maf)
names(new_release)

## Quick adding

Here, rather than rerunning the entire analysis, because we know we are adding only WES samples, we can download the previous release's MAF, add the samples, update any annotations, and perform any global filters at the end.

First we need to do an additional step of filtering on coverage and number 

In [ ]:
%%R
previous.release.maf <- load.from.taiga(data.name='depmap-mutations-maf-35fe', data.file=paste0('mutations.',prevname),data.version=9)
if (colnames(previous.release.maf)[1] == 'X1' || colnames(previous.release.maf)[1] == "") {
 previous.release.maf[,1] <- NULL 
}
names(previous.release.maf)

In [ ]:
%%R
merged <- addToMainMutation(previous.release.maf, new_release)

In [ ]:
%%R
## Adding more
##more <- load.from.taiga(data.name='',  data.version=3, data.file='')
##if (colnames(previous.release.maf)[1] == 'X1' || colnames(previous.release.maf)[1] == '') {
## previous.release.maf[,1] <- NULL 
##}
##merged <- addToMainMutation(merged, more)

In [ ]:
%%R
names(merged)

In [ ]:
%%R
## check if some rows have nans
length(which(is.na(merged$Hugo_Symbol)))

In [ ]:
%%R
dim(merged)

In [ ]:
%%R
filtered <- filterAllelicFraction(merged)

In [ ]:
%%R
filtered <- filterMinCoverage(filtered$merged, filtered$removed_from_maf)

In [ ]:
%%R
merged %<>% dplyr::rename(Tumor_Seq_Allele2=Tumor_Seq_Allele1)

In [ ]:
%%R
clean_annotations <- mergeAnnotations(merged,previous.release.maf)

In [ ]:
%%R
# Guillaume's version
new_release <- addAnnotation(filtered$merged, clean_annotations, colnames(previous.release.maf))
# Allie's version
new_release <- maf_add_variant_annotations(new_release)

In [ ]:
%%R
# some matric files that does get used internaly and might be useful
damaging_mutation <- mutation_maf_to_binary_matrix(new_release, damaging =  TRUE)
other_mutation <- mutation_maf_to_binary_matrix(new_release, other = TRUE)
hotspot_mutation <- mutation_maf_to_binary_matrix(new_release, hotspot = TRUE)

In [ ]:
%%R
# Save the ready to upload file to upload to taiga
write.table(
  new_release, 
  paste0('temp/mutations.', release, '.tsv'), sep = '\t', quote = F, row.names = F)
# Save the ready to upload file to upload to taiga
write.table(
  damaging_mutation, 
  paste0('temp/damaging_mutation.', release, '.tsv'), sep = '\t', quote = F, row.names = F)
# Save the ready to upload file to upload to taiga
write.table(
  other_mutation, 
  paste0('temp/other_mutation.', release, '.tsv'), sep = '\t', quote = F, row.names = F)
# Save the ready to upload file to upload to taiga
write.table(
  hotspot_mutation, 
  paste0('temp/hotspot_mutation.', release, '.tsv'), sep = '\t', quote = F, row.names = F)

In [ ]:
a = pd.read_csv('temp/mutations.'+ release+ '.tsv',sep='\t')

In [ ]:
a[a.DepMap_ID=="ACH-002476"]

# Validation

## Compare to previous release

I would run some checks here comparing the results to the previous releases MAF. Namely:

- Count the total number of mutations per cell line, split by type (SNP, INS, DEL)
- Count the total number of mutations observed by position (group by chromosome, start position, end position and count the number of mutations)
- Look at specific differences between the two MAFs (join on DepMap_ID, Chromosome, Start position, End position, Variant_Type). I would do this for WES only

In [ ]:
mutations = pd.read_csv('temp/mutations.'+release+'.tsv', sep='\t')
damaging_mutation = pd.read_csv('temp/damaging_mutation.'+release+'.tsv', sep='\t')
other_mutation = pd.read_csv('temp/other_mutation.'+release+'.tsv', sep='\t')
hotspot_mutation = pd.read_csv('temp/hotspot_mutation.'+release+'.tsv', sep='\t')

## Remove less used 

In [ ]:
len(mutations.columns)

In [ ]:
mutations[mutations["Hugo_Symbol"]=="ACOT4"][mutations['Start_position']==74058831]

In [ ]:
ac_data = mutations[[val for val in mutations.columns.values if '_AC' in val]]

In [ ]:
ac_names = ac_data.columns.values
ac_data = ac_data.values

In [ ]:
ac_data.shape[0]

## check important mutations

In [ ]:
# check MOLM13, MV411 cell lines- The well known mutation status of FLT3

In [ ]:
# check TP53 mutation 

In [ ]:
toofew = 0
allnan = 0
for pos, val in enumerate(ac_data):
    i = 0
    print(100*pos/ac_data.shape[0],end='\r')
    for p, v in enumerate(val):
        if v is np.nan:
            i+=1
    if i==7:
        mutations = mutations.drop[pos]
        allnan+=1

### basic counts

In [ ]:
#Count the total number of mutations per cell line, split by type (SNP, INS, DEL)

In [ ]:
# Count the total number of mutations observed by position

Are mutation consistent?

In [ ]:
#  to check this, if you group all the mutations in the mutations table by Chromosome, Start_position, End_position, Reference_Allele, Tumor_Seq_Allele1 columns, they should all have the same annotation for the other columns (protein change, exac_af, etc...)

QC mutations, for a known dependency, check if it matches mutation of this gene. (if P53 is mutated, cannot have dependency on P53 or MDM2 MDM4/ inverse fir BRAF and KRAF to themselves)

In [ ]:
prevprev= set(tc.get(name='depmap-mutation-calls-9be3', file="depmap_"+prevprevname+"_mutation_calls", version=13).index.tolist())

# uploading on taiga

In [ ]:
gsheets = sheets.get(sheeturl).sheets[6].to_frame()
wes_dmc_embargo = [i for i in gsheets['WES_DMC_embargo'].values.tolist() if i is not np.nan]
blacklist = [i for i in gsheets['blacklist'].values.tolist() if i is not np.nan]

In [ ]:
tc.update_dataset(dataset_permaname="depmap-mutations-maf-35fe",
                 upload_file_path_dict={'temp/mutations.'+release+'.tsv': 'TableTSV'}, 
                 dataset_description="""
# Mutations

filtered and unfiltered mutation files from Broad WES and Sanger WES data mapped to hg19
The MAF file for DepMap that includes all of the latest WES samples. This MAF is generated by merging CCLE (WGS, RNAseq, RD, HC) and Sanger (WES) data.

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal. Must use subsetted dataset instead. These data will not make it on the portal starting 19Q1. With the DMC portal, there is new cell line release prioritization as to which lines can be included, so a new taiga dataset will be created containing CN for the portal.

version 1:  In 19Q1 the WES_AC column has been replaced by two columns, VA_WES_AC and CGA_WES_AC. We are currently using the Van Allen and CGA based pipeline to generate mutation calls. The CGA pipeline includes more filtering on the MAFs than VA and has a better INDEL caller. However, some of these filters may be removing some variants of interest that are still capture by the VA pipeline, which is why both a retained for now. DEPRECATED:  Missing the VA_WES_AC, CGA_WES_AC columns
version 2: 19Q1 data
version 3: 19Q2 data. We are no longer using the CCLE_WES_AC column. We are only using the CGA pipeline for mutation calls.
version 4: Updating to 19Q3interim DEPRECATED
version 5: Updating to 19Q3interim DEPRECATED
version 6: Updating to 19Q3interim
version 7: Updating to 19Q3 DEPRECATED
version 8: reparing the missing mutation problem DEPRECATED
version 9: reparing the missing column problem


## ** Internal 19Q4****

Adding 52 new cell lines. 
Some cells lines have been flagged as:

 - having bad looking copy ration plots = 
 - Genes having a similar CN value accross all []


genes (gene rpkm):
__Rows__:
__Columns__:
Counts (gene counts):
__Rows__:
__Columns__:
Gene level CN data:
__Rows__:
__Columns__:
 DepMap cell line IDs
 gene names in the format HGNC\_symbol (Entrez\_ID)
DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean
 """)

## Internal

In [ ]:
mutations[~mutations.index.isin(blacklist)].to_csv('temp/depmap_'+release+'_mutation_calls', sep='\t')
damaging_mutation[~damaging_mutation.index.isin(blacklist)].to_csv('temp/hotspot_mutation', sep='\t')
other_mutation[~other_mutation.index.isin(blacklist)].to_csv('temp/damaging_mutation', sep='\t')
hotspot_mutation[~hotspot_mutation.index.isin(blacklist)].to_csv('temp/other_mutation', sep='\t')

In [ ]:
tc.update_dataset(dataset_permaname="depmap-mutation-calls-9be3",
                 upload_file_path_dict={'temp/depmap_'+release+'_mutation_calls': 'TableTSV',
                                        'temp/damaging_mutation': 'TableTSV',
                                        'temp/other_mutation': 'TableTSV',
                                        'temp/hotspot_mutation': 'TableTSV'},
                 dataset_description="""
# Internal Mutations

Mutation calls for Internal DepMap data

* Version 1 Internal 18Q1*

original source: /xchip/ccle_dist/broad_only/CMAG/mutations/CCLE_depMap_18Q1_maf_20180202.txt
* Version 2-4 Internal 18Q2*

merged mutations and indels file (1,606 cell lines, including CCLE and Sanger WES reanalysis)
original source: /xchip/ccle_dist/broad_only/CMAG/mutations/CCLE_depMap_18q2_maf_20180502.txt
Binary matrices:
damaging: if isDeleterious is true
missense: if isDeleterious is false
hotspot: if missense and either TCGA or COSMIC hotspot
Version 2 contains the MAF file
* Version 5-6 Internal 18Q3*

version 5 deprecated

original source: /xchip/ccle_dist/broad_only/CMAG/mutations/CCLE_depMap_18q3_maf_20180716.txt

Binary matrices:
damaging: if isDeleterious is true
missense: if isDeleterious is false
hotspot: if missense and either TCGA or COSMIC hotspot
Rows: cell line, Broad (arxspan) IDs

Columns: Gene, HGNC symbol (Entrez ID)

MAF file

* Version 7-8 Internal 18Q4*

version 8 just changes a column name in the MAF file from Broad_ID to DepMap_ID

original source: /xchip/ccle_dist/broad_only/CMAG/mutations/CCLE_DepMap_18Q4_maf_20181028.txt

* Version 9-12 Internal 19Q1*

version 12 updates the column name from VA_WES_AC to CCLE_WES_AC

version 11+ uses an updated definition for hotspot mutations

version 12 contains the correct data for 19Q1

* Version 13 Internal 19Q2*

* Version 14-15 Internal 19Q3*

version 15 fixed entrez ids

* Version 16 Internal 19Q4*

adding 35 new cell lines.

*** Variant annotation column ***

MAF file, added column (Variant_annotation) classifying each variant as either silent, damaging, other conserving, or other non-conserving, based on this mapping (old annotation from Variant_Classification column - new annotation):

Silent - silent
Splice_Site - damaging
Missense_Mutation - other non-conserving
Nonsense_Mutation - damaging
De_novo_Start_OutOfFrame - damaging
Nonstop_Mutation - other non-conserving
Frame_Shift_Del - damaging
Frame_Shift_Ins - damaging
In_Frame_Del - other non-conserving
In_Frame_Ins - other non-conserving
Stop_Codon_Del - other non-conserving
Stop_Codon_Ins - other non-conserving
Start_Codon_SNP - damaging
Start_Codon_Del - damaging
Start_Codon_Ins - damaging
5'Flank - other conserving
Intron - other conserving
IGR - other conserving
3'UTR - other conserving
5'UTR - other conserving
Binary matrices:

damaging: if damaging
other: if other conserving or other non-conserving
hotspot: if it is not a silent mutation and is either TCGA or COSMIC hotspot
Rows: cell line, DepMap (arxspan) IDs

Columns: Gene, HGNC symbol (Entrez ID)
""")

In [ ]:
# To add to a virtual dataset
AddToVirtual(virtual_internal, 'depmap-mutation-calls-9be3', [('CCLE_mutations', 'depmap_'+release+'_mutation_calls')])
# To add to a eternal dataset
AddToVirtual('depmap-a0ab', 'depmap-mutation-calls-9be3', [('CCLE_mutations', 'depmap_'+release+'_mutation_calls')])

## DMC

In [ ]:
tc.update_dataset(dataset_permaname="depmap-mutation-calls-dfce",
                 upload_file_path_dict={'temp/depmap_'+release+'_mutation_calls': 'TableTSV',
                                        'temp/damaging_mutation': 'TableTSV',
                                        'temp/other_mutation': 'TableTSV',
                                        'temp/hotspot_mutation': 'TableTSV'},
                 dataset_description="""
# DMC Mutations

* Version 1-5 DMC 19Q1*

version 5 is a one-off portal thing because dmc wanted to be able to plot if a gene has any mutation as one-hot encoded value in the x/y axes of the data explorer It adds the any_mutation matrix, but does not change the others. Code used to generate:

from taigapy import TaigaClient

c = TaigaClient()

dmc_19q1_mutation_taiga_root = "depmap-mutation-calls-dfce.3/"
other_matrix = c.get(dmc_19q1_mutation_taiga_root + "other_mutation")
damaging_matrix = c.get(dmc_19q1_mutation_taiga_root + "damaging_mutation")
hotspot_matrix = c.get(dmc_19q1_mutation_taiga_root + "hotspot_mutation")

df = other_matrix.append(damaging_matrix)
df = df.groupby(level=0).sum()

df = df.append(hotspot_matrix)
df = df.groupby(level=0).sum()

df[df > 1] = 1

df.to_csv('any_mutation.csv')
The code uses version 3 because the dmc portal was using version 3

version 4 updates the column name from VA_WES_AC to CCLE_WES_AC

version 3 has an updated definition for hotspot mutations

version 2+ contains the correct data for 19Q1

* Version 6 DMC 19Q2*

* Version 7-8 DMC 19Q3*

* Version 9 DMC 19Q4*

adding 52 new cell lines.

version 8 fixed entrez ids

MAF file, added column (Variant_annotation) classifying each variant as either silent, damaging, other conserving, or other non-conserving, based on this mapping (old annotation from Variant_Classification column - new annotation):

Silent - silent
Splice_Site - damaging
Missense_Mutation - other non-conserving
Nonsense_Mutation - damaging
De_novo_Start_OutOfFrame - damaging
Nonstop_Mutation - other non-conserving
Frame_Shift_Del - damaging
Frame_Shift_Ins - damaging
In_Frame_Del - other non-conserving
In_Frame_Ins - other non-conserving
Stop_Codon_Del - other non-conserving
Stop_Codon_Ins - other non-conserving
Start_Codon_SNP - damaging
Start_Codon_Del - damaging
Start_Codon_Ins - damaging
5'Flank - other conserving
Intron - other conserving
IGR - other conserving
3'UTR - other conserving
5'UTR - other conserving
Binary matrices:

damaging: if damaging
other: if other conserving or other non-conserving
hotspot: if it is not a silent mutation and is either TCGA or COSMIC hotspot
Rows: cell line, DepMap (arxspan) IDs

Columns: Gene, HGNC symbol (Entrez ID)
 """)

In [ ]:
# To add to a virtual dataset
AddToVirtual(virtual_dmc, 'depmap-mutation-calls-dfce', [('CCLE_mutations', 'depmap_'+release+'_mutation_calls')])

## Public

In [ ]:
mutations = mutations[mutations.index.isin(prevprev)]
mutations[~mutations.index.isin(blacklist+wes_dmc_embargo)].to_csv('temp/depmap_'+release+'_mutation_calls', sep='\t')
damaging_mutation = damaging_mutation[damaging_mutation.index.isin(prevprev)]
damaging_mutation[~damaging_mutation.index.isin(blacklist+wes_dmc_embargo)].to_csv('temp/hotspot_mutation', sep='\t')
other_mutation = other_mutation[other_mutation.index.isin(prevprev)]
other_mutation[~other_mutation.index.isin(blacklist+wes_dmc_embargo)].to_csv('temp/damaging_mutation', sep='\t')
hotspot_mutation = hotspot_mutation[hotspot_mutation.index.isin(prevprev)]
hotspot_mutation[~hotspot_mutation.index.isin(blacklist+wes_dmc_embargo)].to_csv('temp/other_mutation', sep='\t')

In [ ]:
tc.update_dataset(dataset_permaname="depmap-mutation-calls-9a1a",
                 upload_file_path_dict={'temp/depmap_'+release+'_mutation_calls': 'TableTSV',
                                        'temp/damaging_mutation': 'TableTSV',
                                        'temp/other_mutation': 'TableTSV',
                                        'temp/hotspot_mutation': 'TableTSV'},
                 dataset_description="""
# Public Mutations

Mutation calls for Public DepMap data

* Version 1 Public 18Q1*

original source: CCLE data portal
* Version 2 Public 18Q2*

merged mutations and indels file (1,549 cell lines total, including data for 63 newly released cell lines)
original source: /xchip/ccle_dist/public/DepMap_18Q2/CCLE_DepMap_18Q2_maf_20180502.txt
* Version 3-4 Public 18Q3*

version 3 deprecated

original source: /xchip/ccle_dist/public/DepMap_18Q3/CCLE_DepMap_18q3_maf_20180718.txt

Binary matrices:
damaging: if isDeleterious is true
missense: if isDeleterious is false
hotspot: if missense and either TCGA or COSMIC hotspot
Rows: cell line, Broad (arxspan) IDs

Columns: Gene, HGNC symbol (Entrez ID)

MAF file

* Version 5 Public 18Q4*

original source: /xchip/ccle_dist/public/DepMap_18Q4/CCLE_DepMap_18q4_maf_20181029.txt

* Version 6-9 Public 19Q1*

version 9 updates the column name from VA_WES_AC to CCLE_WES_AC

version 8 uses an updated definition for hotspot mutations

version 9 contains the correct data for 19Q1

* Version 10 Public 19Q2*

* Version 11-12 Public 19Q3*

version 12 fixed entrez ids

* Version 13 Public 19Q4*

adding 52 new cell lines

MAF file, added column (Variant_annotation) classifying each variant as either silent, damaging, other conserving, or other non-conserving, based on this mapping (old annotation from Variant_Classification column - new annotation):

Silent - silent
Splice_Site - damaging
Missense_Mutation - other non-conserving
Nonsense_Mutation - damaging
De_novo_Start_OutOfFrame - damaging
Nonstop_Mutation - other non-conserving
Frame_Shift_Del - damaging
Frame_Shift_Ins - damaging
In_Frame_Del - other non-conserving
In_Frame_Ins - other non-conserving
Stop_Codon_Del - other non-conserving
Stop_Codon_Ins - other non-conserving
Start_Codon_SNP - damaging
Start_Codon_Del - damaging
Start_Codon_Ins - damaging
5'Flank - other conserving
Intron - other conserving
IGR - other conserving
3'UTR - other conserving
5'UTR - other conserving
Binary matrices:

damaging: if damaging
other: if other conserving or other non-conserving
hotspot: if it is not a silent mutation and is either TCGA or COSMIC hotspot
Rows: cell line, DepMap (arxspan) IDs

Columns: Gene, HGNC symbol (Entrez ID)
 """)

In [ ]:
# To add to a virtual dataset
AddToVirtual(virtual_public, 'depmap-mutation-calls-9a1a', [('CCLE_mutations', 'depmap_'+release+'_mutation_calls')])